In [1]:
import os

from python_speech_features import mfcc
from python_speech_features import delta
from python_speech_features import logfbank
import scipy.io.wavfile as wav
from random import randint

In [2]:
# every frame is considered independent

no_of_features = 13
no_of_fbank_features = 20
no_of_columns = (3 * no_of_features) + no_of_fbank_features

def get_feature_vectors(dataset_type):
    
    #set parameters for training and testing
    if (dataset_type == "train"):
        directory = os.path.join(os.getcwd(), 'data_thuyg20_sre/enroll')
        no_of_frames = 400
        start_frame = 1
    elif (dataset_type == "test"):    
        directory = os.path.join(os.getcwd(), 'data_thuyg20_sre/test')
        no_of_frames = 40
        start_frame = randint(1, 200)
        
    dataset = numpy.empty([0, no_of_columns + 1])
    
    for file in os.listdir(directory):
        
        # filter speakers
        names = ['F101', 'F102', 'F103', 'F104', 'F105', 'M101', 'M102', 'M103', 'M104']

        if any(name in file for name in names):
            
            # extract mfcc vectors
            (rate,sig) = wav.read(os.path.join(directory, file))
            fbank_feat = logfbank(sig,rate)
            mfcc_feat = mfcc(sig,rate)
            d_mfcc_feat = delta(mfcc_feat, 2)
            dd_mfcc_feat = delta(d_mfcc_feat, 2)
            
#             fbank_feat = logfbank(sig,rate)
            mfcc_vectors = mfcc_feat[start_frame:start_frame+no_of_frames,:no_of_features]
            dmfcc_vectors = d_mfcc_feat[start_frame:start_frame+no_of_frames,:no_of_features]
            ddmfcc_vectors = dd_mfcc_feat[start_frame:start_frame+no_of_frames,:no_of_features]
            fbank_vectors = fbank_feat[start_frame:start_frame+no_of_frames,:no_of_fbank_features]
            
            feature_vectors = numpy.hstack((mfcc_vectors, dmfcc_vectors, ddmfcc_vectors, fbank_vectors))
#             print(feature_vectors.shape)
            
            # get speaker index from filename
            speaker_index = file.split("_")[0]
            if speaker_index[0] == 'M':
                speaker_index = 5 + int(speaker_index[3:])
            else:
                speaker_index = int(speaker_index[3:])

            #append speaker index to feature vectors
            np_speaker_index = numpy.array([speaker_index])
            temp = numpy.tile(np_speaker_index[numpy.newaxis,:], (feature_vectors.shape[0],1))
            concatenated_feature_vector = numpy.concatenate((feature_vectors,temp), axis=1)
            
#             print(concatenated_feature_vector.shape)
#             print(fbank_vectors.shape)
            
            # append file's data to dataset
            dataset = numpy.concatenate((dataset, concatenated_feature_vector), axis=0)
            

    return dataset


In [3]:
from keras.models import Sequential
import numpy as numpy 

Using Theano backend.


In [4]:
# from numpy import genfromtxt
my_data = get_feature_vectors("train")


In [5]:
# print(my_data)
print(my_data.shape)

(3600, 60)


In [6]:
Y = numpy.copy(my_data[:, no_of_columns:])
print(Y.shape)

(3600, 1)


In [7]:
X = numpy.copy(my_data[:, :no_of_columns])
print(X.shape)
mean = X.mean(0, keepdims=True)

print(mean.shape)
std_deviation = numpy.std(X, axis=0, keepdims=True)
print(std_deviation.shape)

normalized_X = (X - mean) / std_deviation
print(normalized_X.shape)


(3600, 59)
(1, 59)
(1, 59)
(3600, 59)


In [8]:
from keras import utils as np_utils
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

one_hot_labels = np_utils.to_categorical(Y, num_classes=10)
print(one_hot_labels)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [9]:
def train(layers):

    # MultiLayer Perceptron
    model = Sequential()

    # 3000 0.6 1500 0.6 -> 57% acc
    # 2560 0.7 1280 0.7 -> 60% acc
    # 2560 0.7 1600 0.7 -> 51% acc
    # 2560 0.6 1600 0.7 -> 49%
    # 2560 0.7 1280 -   -> 50
    # 3000 0.7 1280 0.7 -> 54% acc
    # 2560 0.7 1280 0.8 -> 45% acc

    model.add(Dense(3000, activation='tanh', input_dim=no_of_columns))
#     model.add(Dropout(0.7))
#     model.add(Dense(1280, activation='tanh'))
#     model.add(Dropout(0.7))
    
    for layer in layers:
        model.add(Dense(layer, activation='tanh'))
        model.add(Dropout(0.3))
    
    model.add(Dense(10, activation='softmax'))

    sgd = SGD(lr=0.005, decay=1e-5, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])

    model.fit(normalized_X, one_hot_labels, epochs=10, batch_size=32)
    
    return model
# score = model.evaluate(x_test, y_test, batch_size=128)

In [10]:
def test(model):
    
    test_model = get_feature_vectors("test")
#     print(test_model.shape)

    test_X = numpy.copy(test_model[:, :no_of_columns])
#     print(test_X.shape)

    normalized_test_X = (test_X - mean) / std_deviation
#     print(normalized_test_X.shape)

    test_Y = numpy.copy(test_model[:, no_of_columns:])
#     print(test_Y.shape)
    test_labels = np_utils.to_categorical(test_Y, num_classes=10)

#     print(model.test_on_batch(test_X, test_labels, sample_weight=None))
#     print(model.metrics_names)
    predictions = model.predict(test_X)
    
    b = [sum(predictions[current: current+40]) for current in range(0, len(predictions), 40)]
    predicted_Y = []
    for row in b:
        predicted_Y.append(row.argmax(axis=0))
        
    diff = predicted_Y - test_Y[::40].T[0]
    correct = sum(x == 0 for x in diff)
    total = len(predicted_Y)
    percent = correct/total * 100
    print(str(correct) + " / " + str(total) + " = " + str(percent))
    
    loss = str(numpy.sum(numpy.abs(diff)))
    print("Loss = " + loss)
    
    return correct, percent, loss

In [ ]:
# model = train([i, j, k, l])

In [14]:
# with open('automate_op.csv', 'a') as the_file:
#     the_file.write('Hello\n')

# print("Test", file=f)
for i in range(100, 4001,500):  
    for j in range(100,4001,500):  
        for k in range(100,4001,500):  
            for l in range(100,4001,500):  
#                 print(str(i) + ", " + str(j))
                model = train([i, j, k, l])

                correct, percent, loss = test(model)
                string = "3000, " + str(i) + ", " + str(j) + ", " + str(k) + ", " + str(l) + ", " + str(correct) + ", " + str(percent) + ", " + str(loss)
                f = open('full_automate_op.csv', 'a')
                print(string, file=f)
                f.close()

                correct, percent, loss = test(model)
                string = "3000, " + str(i) + ", " + str(j) + ", " + str(k) + ", " + str(l) + ", " + str(correct) + ", " + str(percent) + ", " + str(loss)
                f = open('full_automate_op.csv', 'a')
                print(string, file=f)
                f.close()

                correct, percent, loss = test(model)
                string = "3000, " + str(i) + ", " + str(j) + ", " + str(k) + ", " + str(l) + ", " + str(correct) + ", " + str(percent) + ", " + str(loss)
                f = open('full_automate_op.csv', 'a')
                print(string, file=f)
                f.close()

Epoch 1/10
3600/3600 [==============================] - 4s 978us/step - loss: 1.9156 - acc: 0.3172
Epoch 2/10
3600/3600 [==============================] - 4s 1ms/step - loss: 1.5064 - acc: 0.4731
Epoch 3/10
3600/3600 [==============================] - 3s 937us/step - loss: 1.3582 - acc: 0.5236
Epoch 4/10
3600/3600 [==============================] - 4s 1ms/step - loss: 1.2713 - acc: 0.5583
Epoch 5/10
3600/3600 [==============================] - 4s 1ms/step - loss: 1.2054 - acc: 0.5839
Epoch 6/10
3600/3600 [==============================] - 4s 1ms/step - loss: 1.1193 - acc: 0.6097
Epoch 7/10
3600/3600 [==============================] - 4s 1ms/step - loss: 1.1042 - acc: 0.6194
Epoch 8/10
3600/3600 [==============================] - 4s 1ms/step - loss: 1.0432 - acc: 0.6342
Epoch 9/10
3600/3600 [==============================] - 5s 1ms/step - loss: 0.9871 - acc: 0.6544
Epoch 10/10
3600/3600 [==============================] - 4s 1ms/step - loss: 0.9479 - acc: 0.6761
57 / 90 = 63.3333333333
L

3600/3600 [==============================] - 5s 1ms/step - loss: 1.0365 - acc: 0.6381
Epoch 8/10
3600/3600 [==============================] - 5s 1ms/step - loss: 0.9947 - acc: 0.6450
Epoch 9/10
3600/3600 [==============================] - 5s 1ms/step - loss: 0.9421 - acc: 0.6694
Epoch 10/10
3600/3600 [==============================] - 5s 1ms/step - loss: 0.9086 - acc: 0.6786
53 / 90 = 58.8888888889
Loss = 120.0
54 / 90 = 60.0
Loss = 118.0
55 / 90 = 61.1111111111
Loss = 107.0
Epoch 1/10
3600/3600 [==============================] - 3s 802us/step - loss: 1.8385 - acc: 0.3447
Epoch 2/10
3600/3600 [==============================] - 3s 815us/step - loss: 1.4304 - acc: 0.5075
Epoch 3/10
3600/3600 [==============================] - 3s 814us/step - loss: 1.2974 - acc: 0.5506
Epoch 4/10
3600/3600 [==============================] - 3s 927us/step - loss: 1.2090 - acc: 0.5750
Epoch 5/10
3600/3600 [==============================] - 3s 748us/step - loss: 1.1331 - acc: 0.6097
Epoch 6/10
3600/3600 [===

3600/3600 [==============================] - 11s 3ms/step - loss: 0.9748 - acc: 0.6561
Epoch 9/10
3600/3600 [==============================] - 11s 3ms/step - loss: 0.9414 - acc: 0.6636
Epoch 10/10
3600/3600 [==============================] - 11s 3ms/step - loss: 0.8997 - acc: 0.6867
49 / 90 = 54.4444444444
Loss = 126.0
52 / 90 = 57.7777777778
Loss = 109.0
52 / 90 = 57.7777777778
Loss = 114.0
Epoch 1/10
3600/3600 [==============================] - 12s 3ms/step - loss: 1.7947 - acc: 0.3647
Epoch 2/10
3600/3600 [==============================] - 12s 3ms/step - loss: 1.3762 - acc: 0.5133
Epoch 3/10
3600/3600 [==============================] - 12s 3ms/step - loss: 1.2548 - acc: 0.5619
Epoch 4/10
3600/3600 [==============================] - 13s 4ms/step - loss: 1.1801 - acc: 0.5853
Epoch 5/10
3600/3600 [==============================] - 13s 4ms/step - loss: 1.1313 - acc: 0.6119
Epoch 6/10
3600/3600 [==============================] - 13s 4ms/step - loss: 1.0525 - acc: 0.6306
Epoch 7/10
3600/3

3600/3600 [==============================] - 15s 4ms/step - loss: 0.9739 - acc: 0.6578
Epoch 9/10
3600/3600 [==============================] - 14s 4ms/step - loss: 0.9122 - acc: 0.6819
Epoch 10/10
3600/3600 [==============================] - 14s 4ms/step - loss: 0.8744 - acc: 0.6886
50 / 90 = 55.5555555556
Loss = 112.0
49 / 90 = 54.4444444444
Loss = 115.0
46 / 90 = 51.1111111111
Loss = 125.0
Epoch 1/10
3600/3600 [==============================] - 17s 5ms/step - loss: 1.7875 - acc: 0.3614
Epoch 2/10
3600/3600 [==============================] - 17s 5ms/step - loss: 1.3706 - acc: 0.5139
Epoch 3/10
3600/3600 [==============================] - 17s 5ms/step - loss: 1.2592 - acc: 0.5611
Epoch 4/10
3600/3600 [==============================] - 17s 5ms/step - loss: 1.1697 - acc: 0.5997
Epoch 5/10
3600/3600 [==============================] - 18s 5ms/step - loss: 1.0992 - acc: 0.6208
Epoch 6/10
3600/3600 [==============================] - 17s 5ms/step - loss: 1.0633 - acc: 0.6261
Epoch 7/10
3600/3

Epoch 3/10
3600/3600 [==============================] - 17s 5ms/step - loss: 1.2531 - acc: 0.5636
Epoch 4/10
3600/3600 [==============================] - 17s 5ms/step - loss: 1.1500 - acc: 0.5986
Epoch 5/10
3600/3600 [==============================] - 17s 5ms/step - loss: 1.1202 - acc: 0.6208
Epoch 6/10
3600/3600 [==============================] - 17s 5ms/step - loss: 1.0529 - acc: 0.6317
Epoch 7/10
3600/3600 [==============================] - 17s 5ms/step - loss: 0.9904 - acc: 0.6558
Epoch 8/10
3600/3600 [==============================] - 17s 5ms/step - loss: 0.9456 - acc: 0.6650
Epoch 9/10
3600/3600 [==============================] - 17s 5ms/step - loss: 0.9184 - acc: 0.6836
Epoch 10/10
3600/3600 [==============================] - 17s 5ms/step - loss: 0.8920 - acc: 0.6917
55 / 90 = 61.1111111111
Loss = 106.0
55 / 90 = 61.1111111111
Loss = 89.0
53 / 90 = 58.8888888889
Loss = 102.0
Epoch 1/10
3600/3600 [==============================] - 21s 6ms/step - loss: 1.8047 - acc: 0.3558
Epoch 2

...

50 / 90 = 55.5555555556
Loss = 126.0
43 / 90 = 47.7777777778
Loss = 146.0
Epoch 1/10
3600/3600 [==============================] - 13s 3ms/step - loss: 1.7388 - acc: 0.3761
Epoch 2/10
3600/3600 [==============================] - 12s 3ms/step - loss: 1.3329 - acc: 0.5422
Epoch 3/10
3600/3600 [==============================] - 12s 3ms/step - loss: 1.1882 - acc: 0.5764
Epoch 4/10
3600/3600 [==============================] - 13s 4ms/step - loss: 1.1339 - acc: 0.6033
Epoch 5/10
3600/3600 [==============================] - 13s 4ms/step - loss: 1.0767 - acc: 0.6250
Epoch 6/10
3600/3600 [==============================] - 13s 4ms/step - loss: 1.0131 - acc: 0.6406
Epoch 7/10
3600/3600 [==============================] - 13s 4ms/step - loss: 0.9306 - acc: 0.6750
Epoch 8/10
3600/3600 [==============================] - 13s 4ms/step - loss: 0.8968 - acc: 0.6936
Epoch 9/10
3600/3600 [==============================] - 13s 4ms/step - loss: 0.8298 - acc: 0.6989
Epoch 10/10
3600/3600 [=====================

3600/3600 [==============================] - 12s 3ms/step - loss: 0.9282 - acc: 0.6689
Epoch 7/10
3600/3600 [==============================] - 12s 3ms/step - loss: 0.8726 - acc: 0.6836
Epoch 8/10
3600/3600 [==============================] - 13s 4ms/step - loss: 0.8196 - acc: 0.7078
Epoch 9/10
3600/3600 [==============================] - 13s 4ms/step - loss: 0.7619 - acc: 0.7239
Epoch 10/10
3600/3600 [==============================] - 12s 3ms/step - loss: 0.7258 - acc: 0.7381
54 / 90 = 60.0
Loss = 89.0
55 / 90 = 61.1111111111
Loss = 105.0
51 / 90 = 56.6666666667
Loss = 104.0
Epoch 1/10
3600/3600 [==============================] - 15s 4ms/step - loss: 1.7284 - acc: 0.3864
Epoch 2/10
3600/3600 [==============================] - 15s 4ms/step - loss: 1.2996 - acc: 0.5453
Epoch 3/10
3600/3600 [==============================] - 16s 4ms/step - loss: 1.2081 - acc: 0.5864
Epoch 4/10
3600/3600 [==============================] - 15s 4ms/step - loss: 1.1101 - acc: 0.6122
Epoch 5/10
3600/3600 [=====

44 / 90 = 48.8888888889
Loss = 121.0
Epoch 1/10
3600/3600 [==============================] - 14s 4ms/step - loss: 1.7437 - acc: 0.3761
Epoch 2/10
3600/3600 [==============================] - 14s 4ms/step - loss: 1.3359 - acc: 0.5350
Epoch 3/10
3600/3600 [==============================] - 14s 4ms/step - loss: 1.1798 - acc: 0.5917
Epoch 4/10
3600/3600 [==============================] - 14s 4ms/step - loss: 1.1054 - acc: 0.6136
Epoch 5/10
3600/3600 [==============================] - 14s 4ms/step - loss: 1.0299 - acc: 0.6289
Epoch 6/10
3600/3600 [==============================] - 14s 4ms/step - loss: 0.9470 - acc: 0.6692
Epoch 7/10
3600/3600 [==============================] - 14s 4ms/step - loss: 0.8973 - acc: 0.6797
Epoch 8/10
3600/3600 [==============================] - 15s 4ms/step - loss: 0.8281 - acc: 0.7058
Epoch 9/10
3600/3600 [==============================] - 14s 4ms/step - loss: 0.7825 - acc: 0.7172
Epoch 10/10
3600/3600 [==============================] - 15s 4ms/step - loss: 0.7

49 / 90 = 54.4444444444
Loss = 115.0
Epoch 1/10
3600/3600 [==============================] - 45s 13ms/step - loss: 1.7156 - acc: 0.4014
Epoch 2/10
 384/3600 [==>...........................] - ETA: 44s - loss: 1.2815 - acc: 0.5625

KeyboardInterrupt: 

In [55]:

    
# print(predicted_Y)
# print(test_Y[::40].T)

# for t, p in zip(test_Y[::40].T[0], predicted_Y):
#     print (int(t), p)

f.close()

In [28]:
# majority

# argmax_pred = numpy.argmax(predictions, axis=1)
# argmax_pred = argmax_pred.reshape((-1, 25))

# from scipy.stats import mode
# argmax_pred = mode(argmax_pred, axis=-1)[0]

# # print(argmax_pred)
# # print(test_Y)

# for t, p in zip(test_Y[::40].T[0], argmax_pred):
#     print (int(t), p)

In [17]:
# def find_majority(k):
#     myMap = {}
#     maximum = ( '', 0 ) # (occurring element, occurrences)
#     for n in k:
#         if n in myMap: myMap[n] += 1
#         else: myMap[n] = 1

#         # Keep track of maximum on the go
#         if myMap[n] > maximum[1]: maximum = (n,myMap[n])

#     return maximum